## Use a SQL database for clinical data 🧪

Demo to quickly load 400k+ drug-disease associations in a MySQL database on the DSRI with Python, and pandas.

In [1]:
# !pip install -r requirements.txt
import mysql.connector as mysql
import pandas as pd 

/home/vemonet/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Download the data

From http://snap.stanford.edu/biodata/datasets/10004/10004-DCh-Miner.html using Pandas.

We could also add `DRUGBANK:` at the start of the Chemical ID to have valid CURIEs (namespace + identifier). But for some reason the apply lead Pandas to shuffle the rows, and consider the header as row! Even if the `print` shows that the header. This is a major bug in Pandas, be careful as it can easily lead to error in the data.

In [2]:
data = pd.read_csv('https://snap.stanford.edu/biodata/datasets/10004/files/DCh-Miner_miner-disease-chemical.tsv.gz', sep='\t', header=0)
# data["Chemical"] = data["Chemical"].apply (lambda row: 'DRUGBANK:' + row)
print(data)
data.to_csv('mined-disease-chemical-associations.csv', index=False, header=False)

       # Disease(MESH)          Chemical
0         MESH:D005923  DRUGBANK:DB00564
1         MESH:D009503  DRUGBANK:DB01072
2         MESH:D016115  DRUGBANK:DB01759
3         MESH:D018476  DRUGBANK:DB00451
4         MESH:C567059  DRUGBANK:DB00641
...                ...               ...
466652    MESH:C565545  DRUGBANK:DB00482
466653    MESH:D009164  DRUGBANK:DB00977
466654    MESH:D010518  DRUGBANK:DB04216
466655    MESH:D002653  DRUGBANK:DB02701
466656    MESH:D013119  DRUGBANK:DB04465

[466657 rows x 2 columns]


## Load the data in the database

Connect to the MySQL database using the database Service name (chosen when starting the database), and create the table for drug-disease associations in the default database selected by postgres


In [3]:
try: 
    conn.close()
except:
    print('No conn to close')
conn = mysql.connect(
    host='mysql-demo',
    # host='localhost',
    database='sampledb',
    user='root',
    password='favdbpass',
    autocommit=True,
    allow_local_infile=True)
cursor = conn.cursor()
cursor.execute("SET GLOBAL local_infile = 1;")
cursor.execute("SET GLOBAL max_allowed_packet=1073741824;")
cursor.execute("DROP TABLE IF EXISTS associations;")
cursor.execute("""CREATE TABLE associations(
   disease_id VARCHAR(255),
   drug_id VARCHAR(255),
   PRIMARY KEY (disease_id, drug_id)
);""")

Load the CSV file in the database `associations` table.

It can also be done with Pandas (if the stars are properly aligned in the sky):

```
data.to_sql('associations', conn, if_exists='replace', index=False)
```

In [5]:
# You need to use absolute path

cursor.execute("""LOAD DATA LOCAL INFILE '/home/jovyan/dsri-demo/mined-disease-chemical-associations.csv'
    INTO TABLE associations;
""")

In [4]:
## Or upload row by row
# for i, row in data.iterrows():
#     print(row['Chemical'])
#     print(row['# Disease(MESH)'])
#     cursor.execute ("INSERT INTO associations (disease_id, chemical_id) VALUES ('" + row['# Disease(MESH)'] + "', '" + row['Chemical'] + "');")
#     break

## Query the database

You can now run a `SELECT` query:

In [6]:
print('Number of associations in the database:')
cursor.execute('SELECT COUNT(*) FROM associations;')
records = cursor.fetchall()
for i in records:
    print(i)
print('\nSample of associations in the database:')
cursor.execute('SELECT disease_id, drug_id FROM associations LIMIT 3;')
records = cursor.fetchall()
for i in records:
    print(i)
conn.close()

Number of associations in the database:
(466657,)

Sample of associations in the database:
('MESH:D005923', 'DRUGBANK:DB00564')
('MESH:D009503', 'DRUGBANK:DB01072')
('MESH:D016115', 'DRUGBANK:DB01759')


## What's next?

* You can also connect to the MySQL database using the terminal:

    ```
    sudo apt install mysql
    mysql -h mysql-demo -U root sampledb
    ```
* Setup a user interface to manage the database
* Take a look into visualization tools to explore your database, such as Apache Superset.